### 1) (***) Tenemos información sobre recetas en 3 RDD de Spark.
Recetas: (id_receta, nombre, tiempo_preparación, dificultad)

Ingredientes: (id_ingrediente, nombre)

Ingredientes por Receta: (id_receta, id_ingrediente, cantidad)
#### Se pide:
a) Obtener el nombre de todas las
recetas que tengan Cordero. (7 puntos)

b) Calcular la cantidad total de cada
ingrediente si queremos hacer todas las
recetas con Cordero que sean fáciles. (8
puntos) 

In [45]:
import pandas as pd

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
recetas = [
    (1, "milanesas con ensalada", 60, "facil"),
    (4, "sushi", 100, "difícil"),
    (5, "asado", 110, "facil" ),
    (7, "cordero a la cerveza", 60, "facil"),
    (8, "cordero al disco", 130, "dificil")
]

In [14]:
ingredientes = [
    (1, "pescado"),
    (2, "cordero"),
    (3, "cerveza"),
    (4, "morron"),
    (5, "sal"),
    (6, "vino tinto"),
    (7, "lechuga"),
    (8, "salsa")
]

In [5]:
ingred_x_receta = [
    (5, 2, 1000),
    (5, 5, 10),
    (4, 1, 500),
    (1, 7, 100),
    (7, 2, 500),
    (7, 3, 1000),
    (8, 2, 800),
    (8, 4, 100)
]

In [6]:
recetas_rdd = sc.parallelize(recetas)
ingred_recetas_rdd = sc.parallelize(ingred_x_receta)
ingredientes_rdd = sc.parallelize(ingredientes)

In [9]:
ing = ingredientes_rdd.filter(lambda x: x[1] == "cordero" )
ing_x_re = ingred_recetas_rdd.map(lambda x: (x[1], x[0]))
join1 = ing.join(ing_x_re)
join1.collect()

[(2, ('cordero', 5)), (2, ('cordero', 7)), (2, ('cordero', 8))]

In [10]:
recetas_id = join1.map(lambda x: (x[1][1], x[1][0]))
recetas = recetas_rdd.map(lambda x: (x[0], x[1]))
join2  = recetas_id.join(recetas)
join2.collect()

[(5, ('cordero', 'asado')),
 (7, ('cordero', 'cordero a la cerveza')),
 (8, ('cordero', 'cordero al disco'))]

In [12]:
fin = join2.map(lambda x: x[1][1])
fin.collect()

['asado', 'cordero a la cerveza', 'cordero al disco']

In [13]:
# parte b

In [38]:
facil = recetas_rdd.filter(lambda x: x[3] == "facil").map(lambda x: (x[0], None))
ingredientes = ingred_recetas_rdd.map(lambda x: (x[0], (x[1],x[2])))
join3 = facil.join(ingredientes).map(lambda x: ((x[0], x[1][1][0]), x[1][1][1]))
join3.collect() # receta_id, (ingrediente_id, cant)

[((1, 7), 100), ((5, 2), 1000), ((5, 5), 10), ((7, 2), 500), ((7, 3), 1000)]

In [37]:
ing_name = ingred_recetas_rdd.map(lambda x: (x[1], x[0])).join(ingredientes_rdd)\
                            .map(lambda x: ((x[1][0], x[0]), x[1][1])) # ing, rec
                        
ing_name.collect()

[((4, 1), 'pescado'),
 ((5, 2), 'cordero'),
 ((7, 2), 'cordero'),
 ((8, 2), 'cordero'),
 ((7, 3), 'cerveza'),
 ((8, 4), 'morron'),
 ((5, 5), 'sal'),
 ((1, 7), 'lechuga')]

In [43]:
join4 = join3.join(ing_name).map(lambda x: (x[1][1], x[1][0])).reduceByKey(lambda x,y: x+y)
join4.collect()

[('cordero', 1500), ('sal', 10), ('cerveza', 1000), ('lechuga', 100)]

# Pandas
2) (***) Dada la exitosa convocatoria de los Juegos Olímpicos de la Juventud por parte del público, sus organizadores realizan distintos análisis para planificar las jornadas finales del certamen. Es por ello que cuentan con información en los siguientes archivos csv:

eventos.csv (id_evento, fecha, id_locacion, nombre_evento, id_categoria_deportiva, cantidad_espectadores)

locacion.csv (id_locacion, nombre, capacidad, capacidad_extendida, sede, latitud, longitud)

categorias_deportivas.csv (id_categoria_deportiva, nombre, año_de_adopcion)

El primer archivo cuenta con información de los eventos, indicando la fecha (en formato “YYYY-mm-dd hh:mm:ss”), el lugar donde ocurrió (id_locacion) y la cantidad de espectadores que asistieron. Además se aporta información sobre la categoría deportiva a la cual pertenece el evento. Por otro lado se tiene información sobre las distintas locaciones en la sedes del certamen en las que ocurrieron los eventos. Contamos con información de su capacidad total de espectadores así como de su capacidad extendida (cuantos asientos extras se pueden brindar sobre la capacidad de la locación).
## Se desea obtener:

a) Nombre de la sede que acumuló la mayor cantidad de espectadores en eventos durante el certamen del 14 al 15 de octubre inclusive. Esto es de vital importancia para distribuir el merchandising oficial del evento, para las fechas finales. (7 pts)

b) Nombre del evento y nombre de la categoría deportiva de aquellos eventos cuya cantidad de espectadores superó la capacidad de la locación, más allá de la capacidad extendida. Esto es de vital importancia para detectar problemas de seguridad o si es necesario realizar algún cambio de locación. (8 pts) 

In [46]:
eventos_data = {
        'fecha': ["2019-10-14 15:10:13","2019-10-15 15:10:13", "2019-10-16 15:10:13", "2019-09-14 15:10:13", "2019-10-14 16:10:13"],
        'id_locacion' : [1,1,2,3,4],
        'cantidad_espectadores': [1000, 1500, 3580, 115000, 383],
}

eventos = pd.DataFrame(eventos_data, columns = ['fecha', 'id_locacion', 'cantidad_espectadores'])

In [47]:
locacion_data = {               #cada locacion seria un estadio y la sede es la ciudad donde esta esa locacion
    'id_locacion': [1,2,3,4],
    'sede' : ['Nueva York', 'Nueva York', 'Miami', 'Nevada']
}

locacion = pd.DataFrame(locacion_data, columns = ['id_locacion', 'sede'])

In [64]:
eventos['fecha'] = pd.to_datetime(eventos['fecha'], errors = 'coerce') 
ev = eventos.loc[ eventos['fecha'].dt.day.isin([14,15]) &  (eventos['fecha'].dt.month == 10)]

In [65]:
ev.head()

,fecha,id_locacion,cantidad_espectadores
0,2019-10-14 15:10:13,1,1000
1,2019-10-15 15:10:13,1,1500
4,2019-10-14 16:10:13,4,383


In [72]:
ev.merge(locacion, on='id_locacion').groupby('sede').agg({'cantidad_espectadores': 'sum'})\
            .reset_index().nlargest(1, columns = 'cantidad_espectadores')

,sede,cantidad_espectadores
1,Nueva York,2500


In [99]:
eventos_b_data = {
    'nombre_evento' : ['Evento1', 'Evento2', 'Evento3', 'Evento4'],
    'cantidad_espectadores' : [1000, 3000, 1300, 1700],
    'id_categoria_deportiva' : [1,1,3,1],
    'id_locacion' : [1,2,1,3]    
}

eventos_b = pd.DataFrame(eventos_b_data, columns = ['nombre_evento','cantidad_espectadores','id_categoria_deportiva','id_locacion'])

locacion_b_data = {
    'id_locacion': [1,2,3,4],
    'sede' : ['Nueva York', 'Nueva York', 'Miami', 'Nevada'],
    'capacidad_extendida': [5000, 13000, 9500, 24000],
    'capacidad' : [1000, 2000, 1500, 300] 
}

locacion_b = pd.DataFrame(locacion_b_data, columns = ['id_locacion', 'sede', 'capacidad', 'capacidad_extendida'])

categorias_deportivas = {
    'id_categoria_deportiva' : [1,3],
    'nombre' : ['futbol', 'tennis']
}

categorias = pd.DataFrame(categorias_deportivas, columns = ['id_categoria_deportiva', 'nombre'])

In [ ]:
# eventos.csv (id_evento, fecha, id_locacion, nombre_evento, id_categoria_deportiva, cantidad_espectadores)

# locacion.csv (id_locacion, nombre, capacidad, capacidad_extendida, sede, latitud, longitud)

# categorias_deportivas.csv (id_categoria_deportiva, nombre, año_de_adopcion)


In [100]:
total = categorias.merge(eventos_b, on='id_categoria_deportiva').merge(locacion_b, on='id_locacion')
total['total_exp'] = total.groupby('id_categoria_deportiva')['cantidad_espectadores'].transform('sum')

In [103]:
total.loc[total['total_exp'] > total['capacidad_extendida']][['nombre_evento', 'nombre']]

,nombre_evento,nombre
0,Evento1,futbol
